[![Roboflow Notebooks](https://media.roboflow.com/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://github.com/roboflow/notebooks)

# How to Train YOLO11 Object Detection on a Custom Dataset

---

[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/ultralytics/ultralytics)

YOLO11 builds on the advancements introduced in YOLOv9 and YOLOv10 earlier this year, incorporating improved architectural designs, enhanced feature extraction techniques, and optimized training methods.

YOLO11m achieves a higher mean mAP score on the COCO dataset while using 22% fewer parameters than YOLOv8m, making it computationally lighter without sacrificing performance.

YOLOv11 is available in 5 different sizes, ranging from `2.6M` to `56.9M` parameters, and capable of achieving from `39.5` to `54.7` mAP on the COCO dataset.

## Setup

### Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [1]:
!nvidia-smi

Wed Nov 13 06:49:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLO11 via Ultralytics

In [3]:
%pip install ultralytics supervision roboflow
import ultralytics
ultralytics.checks()

Ultralytics 8.3.29 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.1/112.6 GB disk)


## Fine-tune YOLO11 on custom dataset

**NOTE:** When training YOLOv11, make sure your data is located in `datasets`. If you'd like to change the default location of the data you want to use for fine-tuning, you can do so through Ultralytics' `settings.json`. In this tutorial, we will use one of the [datasets](https://universe.roboflow.com/liangdianzhong/-qvdww) available on [Roboflow Universe](https://universe.roboflow.com/). When downloading, make sure to select the `yolov11` export format.

In [10]:
!unzip /content/annotationDatasetv1.zip

Archive:  /content/annotationDatasetv1.zip
replace annotationDatasetv1/classes.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: annotationDatasetv1/classes.txt  
replace annotationDatasetv1/data.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: annotationDatasetv1/data.yaml  
replace annotationDatasetv1/notes.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace annotationDatasetv1/notes.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: annotationDatasetv1/notes.json  
   creating: annotationDatasetv1/train/
   creating: annotationDatasetv1/train/images/
  inflating: annotationDatasetv1/train/images/03aa9e5f-6.jpg  
  inflating: annotationDatasetv1/train/images/044e1fdc-1.jpg  
  inflating: annotationDatasetv1/train/images/070a57df-37.jpg  
  inflating: annotationDatasetv1/train/images/0cbd269f-19.jpg  
  inflating: annotationDatasetv1/train/images/10bb4d25-16.jpg  
  inflating: annotationDatasetv1/train/images/11d30aa4-23.jpg  
 

In [5]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11s.pt")

100%|██████████| 18.4M/18.4M [00:00<00:00, 105MB/s] 


## Custom Training

In [11]:
%cd {HOME}

!yolo task=detect mode=train model=yolo11s.pt data=/content/annotationDatasetv1/data.yaml epochs=10 imgsz=640 plots=True

/content
Ultralytics 8.3.29 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/annotationDatasetv1/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolo11s.pt data=/content/minimart.v6i.yolov11/data.yaml epochs=10 imgsz=640 plots=True

## Validate fine-tuned model

In [15]:
!yolo task=detect mode=val model=/content/runs/detect/train5/weights/best.pt data=/content/annotationDatasetv1/data.yaml

Ultralytics 8.3.29 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11s summary (fused): 238 layers, 9,424,797 parameters, 0 gradients, 21.4 GFLOPs
val: Scanning /content/annotationDatasetv1/labels.cache... 155 images, 0 backgrounds, 0 corrupt: 100% 155/155 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 10/10 [00:04<00:00,  2.27it/s]
                   all        155        155      0.912      0.937      0.971      0.943
         biskclub_choc          5          5      0.818      0.907      0.962      0.872
       biskclub_orange          5          5      0.905          1      0.995      0.888
    biskclub_pineapple          5          5       0.91          1      0.995      0.968
   biskclub_strawberry          5          5      0.792          1      0.995      0.963
chipsmore_mini_doublechoc          5          5      0.975          1      0.995      0.995
chipsmore_mini_hazelnut          5          

In [ ]:
import locale

print(locale.getpreferredencoding())

import locale
def getpreferredencoding(do_setlocale = True):
 return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!zip -r modelv4.zip /content/runs/detect/train8

UTF-8
  adding: content/runs/detect/train8/ (stored 0%)
  adding: content/runs/detect/train8/train_batch1.jpg (deflated 18%)
  adding: content/runs/detect/train8/val_batch1_labels.jpg (deflated 11%)
  adding: content/runs/detect/train8/labels.jpg (deflated 42%)
  adding: content/runs/detect/train8/train_batch0.jpg (deflated 20%)
  adding: content/runs/detect/train8/confusion_matrix.png (deflated 17%)
  adding: content/runs/detect/train8/events.out.tfevents.1731400700.ba566053b755.7849.0 (deflated 93%)
  adding: content/runs/detect/train8/args.yaml (deflated 53%)
  adding: content/runs/detect/train8/results.png (deflated 7%)
  adding: content/runs/detect/train8/weights/ (stored 0%)
  adding: content/runs/detect/train8/weights/last.pt (deflated 8%)
  adding: content/runs/detect/train8/weights/best.pt (deflated 8%)
  adding: content/runs/detect/train8/val_batch0_labels.jpg (deflated 8%)
  adding: content/runs/detect/train8/P_curve.png (deflated 12%)
  adding: content/runs/detect/train8/R_

## 🏆 Congratulations

### Learning Resources

Roboflow has produced many resources that you may find interesting as you advance your knowledge of computer vision:

- [Roboflow Notebooks](https://github.com/roboflow/notebooks): A repository of over 20 notebooks that walk through how to train custom models with a range of model types, from YOLOv7 to SegFormer.
- [Roboflow YouTube](https://www.youtube.com/c/Roboflow): Our library of videos featuring deep dives into the latest in computer vision, detailed tutorials that accompany our notebooks, and more.
- [Roboflow Discuss](https://discuss.roboflow.com/): Have a question about how to do something on Roboflow? Ask your question on our discussion forum.
- [Roboflow Models](https://roboflow.com): Learn about state-of-the-art models and their performance. Find links and tutorials to guide your learning.

### Convert data formats

Roboflow provides free utilities to convert data between dozens of popular computer vision formats. Check out [Roboflow Formats](https://roboflow.com/formats) to find tutorials on how to convert data between formats in a few clicks.

### Connect computer vision to your project logic

[Roboflow Templates](https://roboflow.com/templates) is a public gallery of code snippets that you can use to connect computer vision to your project logic. Code snippets range from sending emails after inference to measuring object distance between detections.